In [200]:
import numpy as np
import sys
sys.path.insert(1, './fashion-mnist/utils')
import mnist_reader

In [201]:
x_train, y_train = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='train')
x_test, y_test = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='t10k')
print('x shape:', x_train.shape)
print('y shape:', y_train.shape)

x shape: (60000, 784)
y shape: (60000,)


In [202]:
# Normalizar datos, 255 es el valor maximo de RGB
x_train = x_train / 255

In [203]:
# Transformar vector y, para reflejar nuestras 10 neuronas de salida
y_trains = []
for i in y_train:
    y = []
    for j in range(0, 10):
        if i == j:
            y.append(1.0)
        else:
            y.append(0.0)
    y_trains.append(y)

y_trains = np.array(y_trains)
y_trains.shape

(60000, 10)

In [204]:
class NeuralNet:
    def __init__(self, x, y):
        np.random.seed(69)
        self.x = x
        self.y = y
        self.learning_rate = 0.01
        n = 1 / np.sqrt(x.shape[1])
        self.w_ih = np.random.uniform(low=-n, high=n, size=(x.shape[1] // 3, x.shape[1]))
        self.w_ho = np.random.uniform(low=-n, high=n, size=(y.shape[1], x.shape[1] // 3))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
        
    def feed_forward(self):
        # Capa input a hidden
        z_ih = np.matmul(self.w_ih, self.x.T)
        self.a_ih = self.sigmoid(z_ih)
        
        # Capa hidden a output
        z_ho = np.matmul(self.w_ho, self.a_ih)
        self.a_ho = self.sigmoid(z_ho)
        
    def back_propagate(self):
        error_o = self.y.T - self.a_ho
        error_od = error_o * (self.a_ho * (1.0 - self.a_ho))

        ho_error = np.matmul(self.w_ho.T, error_od)
        ho_d = ho_error * (self.a_ih * (1.0 - self.a_ih))

        self.w_ih += self.learning_rate * np.matmul(ho_d, self.x)
        self.w_ho += self.learning_rate * np.matmul(error_od, self.a_ih.T)
        
    def train(self, epochs=1000):
        for i in range(0, epochs):
            self.feed_forward()
            self.back_propagate()
            
    def predict(self, x):
        z_ih = np.matmul(self.w_ih, x.T)
        self.a_ih = self.sigmoid(z_ih)
        z_ho = np.matmul(self.w_ho, self.a_ih)
        return self.sigmoid(z_ho)

In [215]:
x_train = x_train[0:100]
y_trains = y_trains[0:100]

net = NeuralNet(x_train, y_trains)
net.train(epochs=100000)

In [216]:
correct = 0
wrong = 0
for x in range(len(x_train)):
    y_pred = net.predict(x_test[x])
    highest = 0.0
    highest_index = 0
    for i in range(len(y_pred)):
        if (y_pred[i] > highest):
            highest = y_pred[i]
            highest_index = i
    if highest_index == y_test[x]:
        correct += 1
    else:
        wrong += 1
        
print('Correct:', correct)
print('Wrong:', wrong)

Correct: 71
Wrong: 29


/Users/sebdev6992/Documents/uvg/2020/s1/ai/lab2/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':
